# Setup


In [1]:
import os
from functools import reduce
from typing import List

import dask
import distributed
import kagglehub
import numpy as np
from dask import dataframe as dd
from dotenv import load_dotenv
from tqdm import tqdm

# blocksize and partition size for dask dataframe
# https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html
# https://docs.dask.org/en/stable/dataframe-create.html#read-from-csv
DASK_BLOCKSIZE = "64MB"  # 64MB seems to be the best
# name of the environment variable stored in ./.env for your sql uri
# easily switch to cloud server here e.g. "URI_AZURE"
ENV_SQL_URI = "URI_PG"
# chunksize for to_sql. reduce value if you run out of memory
CHUNKSIZE = 10000
# path to export data
EXPORT_PATH = "./.export"

In [2]:
# Load environment variables from ./.env
is_dotenv = load_dotenv()
if not is_dotenv:  # remind user to create a .env
    raise Exception(f"Please create a .env and define the {ENV_SQL_URI} variable")
# check if sql uri exists
sql_uri = str(os.getenv(ENV_SQL_URI))
if sql_uri == "None":
    raise Exception(f"Please define an sql uri on key {ENV_SQL_URI}")

In [ ]:
# Set temporary directory so it doesn't burn out my SSD
dask.config.set({"temporary_directory": "./.tmp"})  # type: ignore

In [ ]:
# Start the dask distributed client
# click on the link in output to see the dashboard
distributed.Client()

# Extract


## Download dataset


In [5]:
# Download dataset to project folder
# https://github.com/Kaggle/kagglehub/issues/175
os.environ["KAGGLEHUB_CACHE"] = "./.kaggle"
dataset_path = kagglehub.dataset_download("martinellis/nhl-game-data")

## Define dtypes


In [6]:
# Define column dtypes for each file

# dtype shortcuts
DTYPE_BOOL = "bool[pyarrow]"
DTYPE_INT = "int64[pyarrow]"  # note that pyarrow ints are nullable
DTYPE_INT8 = "int8[pyarrow]"
DTYPE_INT16 = "int16[pyarrow]"
DTYPE_INT32 = "int32[pyarrow]"
DTYPE_FLOAT = "float64[pyarrow]"
DTYPE_STRING = "string[pyarrow]"
DTYPE_DATETIME = "timestamp[s][pyarrow]"

# full definitions
dtype_defs = {}
dtype_defs["game.csv"] = {
    "game_id": DTYPE_INT,
    "season": DTYPE_INT,
    "type": DTYPE_STRING,
    "date_time_GMT": DTYPE_DATETIME,
    "away_team_id": DTYPE_INT8,
    "home_team_id": DTYPE_INT8,
    "away_goals": DTYPE_INT8,
    "home_goals": DTYPE_INT8,
    "outcome": DTYPE_STRING,
    "home_rink_side_start": DTYPE_STRING,
    "venue": DTYPE_STRING,
    "venue_link": DTYPE_STRING,
    "venue_time_zone_id": DTYPE_STRING,
    "venue_time_zone_offset": DTYPE_INT8,
    "venue_time_zone_tz": DTYPE_STRING,
}
dtype_defs["game_goalie_stats.csv"] = {
    "game_id": DTYPE_INT,
    "player_id": DTYPE_INT32,
    "team_id": DTYPE_INT8,
    "timeOnIce": DTYPE_INT,
    "assists": DTYPE_INT,
    "goals": DTYPE_INT,
    "pim": DTYPE_INT,
    "shots": DTYPE_INT,
    "saves": DTYPE_INT,
    "powerPlaySaves": DTYPE_INT,
    "shortHandedSaves": DTYPE_INT,
    "evenSaves": DTYPE_INT,
    "shortHandedShotsAgainst": DTYPE_INT,
    "evenShotsAgainst": DTYPE_INT,
    "powerPlayShotsAgainst": DTYPE_INT,
    "decision": DTYPE_STRING,
    "savePercentage": DTYPE_FLOAT,
    "powerPlaySavePercentage": DTYPE_FLOAT,
    "evenStrengthSavePercentage": DTYPE_FLOAT,
}
dtype_defs["game_goals.csv"] = {
    "play_id": DTYPE_STRING,
    "strength": DTYPE_STRING,
    "gameWinningGoal": DTYPE_BOOL,
    "emptyNet": DTYPE_BOOL,
}
dtype_defs["game_officials.csv"] = {
    "game_id": DTYPE_INT,
    "official_name": DTYPE_STRING,
    "official_type": DTYPE_STRING,
}
dtype_defs["game_penalties.csv"] = {
    "play_id": DTYPE_STRING,
    "penaltySeverity": DTYPE_STRING,
    "penaltyMinutes": DTYPE_INT,
}
dtype_defs["game_plays.csv"] = {
    "play_id": DTYPE_STRING,
    "game_id": DTYPE_INT,
    "team_id_for": DTYPE_INT8,
    "team_id_against": DTYPE_INT8,
    "event": DTYPE_STRING,
    "secondaryType": DTYPE_STRING,
    "x": DTYPE_INT8,
    "y": DTYPE_INT8,
    "period": DTYPE_INT8,
    "periodType": DTYPE_STRING,
    "periodTime": DTYPE_INT16,
    "periodTimeRemaining": DTYPE_INT16,
    "dateTime": DTYPE_DATETIME,
    "goals_away": DTYPE_INT8,
    "goals_home": DTYPE_INT8,
    "description": DTYPE_STRING,
    "st_x": DTYPE_INT8,
    "st_y": DTYPE_INT8,
}
dtype_defs["game_plays_players.csv"] = {
    "play_id": DTYPE_STRING,
    "game_id": DTYPE_INT,
    "player_id": DTYPE_INT32,
    "playerType": DTYPE_STRING,
}
dtype_defs["game_scratches.csv"] = {
    "game_id": DTYPE_INT,
    "team_id": DTYPE_INT8,
    "player_id": DTYPE_INT32,
}
dtype_defs["game_shifts.csv"] = {
    "game_id": DTYPE_INT,
    "player_id": DTYPE_INT32,
    "period": DTYPE_INT8,
    "shift_start": DTYPE_INT16,
    "shift_end": DTYPE_INT16,
}
dtype_defs["game_skater_stats.csv"] = {
    "game_id": DTYPE_INT,
    "player_id": DTYPE_INT32,
    "team_id": DTYPE_INT8,
    "timeOnIce": DTYPE_INT16,
    "assists": DTYPE_INT8,
    "goals": DTYPE_INT8,
    "shots": DTYPE_INT8,
    "hits": DTYPE_INT8,
    "powerPlayGoals": DTYPE_INT8,
    "powerPlayAssists": DTYPE_INT8,
    "penaltyMinutes": DTYPE_INT8,
    "faceOffWins": DTYPE_INT8,
    "faceoffTaken": DTYPE_INT8,
    "takeaways": DTYPE_INT8,
    "giveaways": DTYPE_INT8,
    "shortHandedGoals": DTYPE_INT8,
    "shortHandedAssists": DTYPE_INT8,
    "blocked": DTYPE_INT8,
    "plusMinus": DTYPE_INT8,
    "evenTimeOnIce": DTYPE_INT16,
    "shortHandedTimeOnIce": DTYPE_INT16,
    "powerPlayTimeOnIce": DTYPE_INT16,
}
dtype_defs["game_teams_stats.csv"] = {
    "game_id": DTYPE_INT,
    "team_id": DTYPE_INT8,
    "HoA": DTYPE_STRING,
    "won": "bool",
    "settled_in": DTYPE_STRING,
    "head_coach": DTYPE_STRING,
    "goals": DTYPE_INT,
    "shots": DTYPE_INT,
    "hits": DTYPE_INT,
    "pim": DTYPE_INT,
    "powerPlayOpportunities": DTYPE_INT,
    "powerPlayGoals": DTYPE_INT,
    "faceOffWinPercentage": DTYPE_FLOAT,
    "giveaways": DTYPE_INT,
    "takeaways": DTYPE_INT,
    "blocked": DTYPE_INT,
    "startRinkSide": DTYPE_STRING,
}
dtype_defs["player_info.csv"] = {
    "player_id": DTYPE_INT32,
    "firstName": DTYPE_STRING,
    "lastName": DTYPE_STRING,
    "nationality": DTYPE_STRING,
    "birthCity": DTYPE_STRING,
    "primaryPosition": DTYPE_STRING,
    "birthDate": DTYPE_DATETIME,
    "birthStateProvince": DTYPE_STRING,
    "height": DTYPE_STRING,
    "height_cm": DTYPE_FLOAT,
    "weight": DTYPE_INT,
    "shootsCatches": DTYPE_STRING,
}
dtype_defs["team_info.csv"] = {
    "team_id": DTYPE_INT8,
    "franchiseId": DTYPE_INT,
    "shortName": DTYPE_STRING,
    "teamName": DTYPE_STRING,
    "abbreviation": DTYPE_STRING,
    "link": DTYPE_STRING,
}

## Read files


In [ ]:
# Read all csv files from data directory
dfs: dict[str, dd.DataFrame] = {}
for n in os.listdir(dataset_path):
    if n.endswith(".csv"):
        if n in dtype_defs:
            print(f"✅ reading {n} with dtypes")
            fpath = os.path.join(dataset_path, n)
            dfs[n.split(".")[0]] = dd.read_csv(
                fpath,
                blocksize=DASK_BLOCKSIZE,
                dtype=dtype_defs[n],
                engine="pyarrow",
                dtype_backend="pyarrow",
            )
        else:
            print(f"❌ reading {n} without dtypes")
            dfs[n.split(".")[0]] = dd.read_csv(os.path.join(dataset_path, n))

In [ ]:
# Print partitions
for k in dfs:
    print(f"✂️ npartitions {k}: {dfs[k].npartitions}")

In [9]:
# compute to verify types are compatible
# don't need to run this once types are finalized

# for n in dfs:
#     try:
#         dfs[n].compute()
#         print(f"✅ {n} passed compute")
#     except Exception as e:
#         print(f"❌ {n} did not pass compute due to:")
#         print(e)

# Transform


## Drop duplicate rows


In [ ]:
def drop_duplicates(df: dd.DataFrame, subset: str) -> dd.DataFrame:
    """Drop duplicates and return the dataframe"""
    rows_before = df.shape[0].compute()
    df_deduped = df.drop_duplicates(subset=subset)
    rows_after = df_deduped.shape[0].compute()
    print(f"🗑️ dropped {rows_before - rows_after} rows using {subset}")
    return df_deduped


# I have verified that data with the same key are duplicate rows
dfs["game"] = drop_duplicates(dfs["game"], "game_id")
dfs["game_plays"] = drop_duplicates(dfs["game_plays"], "play_id")
dfs["player_info"] = drop_duplicates(dfs["player_info"], "player_id")
dfs["team_info"] = drop_duplicates(dfs["team_info"], "team_id")

## Drop unreferenced data


In [ ]:
# Drop unreferenced data that is unrecoverable
def drop_unreferenced(
    df_foreign: dd.DataFrame,
    df_foreign_col: str,
    df_primary: dd.DataFrame,
    df_primary_col: str,
) -> dd.DataFrame:
    """Drop unreferenced keys and returns the dataframe"""
    rows_before = df_foreign.shape[0].compute()
    # reduce memory usage with partitions
    selections = []
    for i in range(df_primary.npartitions):
        selections.append(
            df_foreign[df_foreign_col].isin(df_primary[df_primary_col].get_partition(i))
        )
    selector = reduce(lambda x, y: x | y, selections)
    df_foreign = df_foreign[selector]
    rows_after = df_foreign.shape[0].compute()
    print(f"🗑️ dropped {rows_before - rows_after} rows on {df_foreign_col}")
    return df_foreign


dfs["game_goalie_stats"] = drop_unreferenced(
    dfs["game_goalie_stats"], "team_id", dfs["team_info"], "team_id"
)
dfs["game_plays_players"] = drop_unreferenced(
    dfs["game_plays_players"], "play_id", dfs["game_plays"], "play_id"
)
dfs["game_skater_stats"] = drop_unreferenced(
    dfs["game_skater_stats"], "team_id", dfs["team_info"], "team_id"
)
dfs["game_teams_stats"] = drop_unreferenced(
    dfs["game_teams_stats"], "team_id", dfs["team_info"], "team_id"
)

In [12]:
# HACK: to_parquet throws an error without this even though the dtypes seem correct
dfs["game_teams_stats"]["won"] = dfs["game_teams_stats"]["won"].astype("bool[pyarrow]")

## Add summary statistics


In [13]:
def gmean(df: dd.DataFrame, cols: List[str]):
    """Calculate the geometric mean across a list of columns `cols` in `df`"""
    return df[cols].prod(axis=1) ** (1 / len(cols))

In [14]:
# Converting 'won' from bool to int allows us to perform mean() and calculate the win rate
dfs["game_teams_stats"]["winRate"] = dfs["game_teams_stats"]["won"].astype(
    "int8[pyarrow]"
)

# Generate average team stats from per-game team stats
dfs["avg_teams_stats"] = (
    dfs["game_teams_stats"].groupby("team_id").mean(numeric_only=True)
)
dfs["avg_teams_stats"] = dfs["avg_teams_stats"].drop("game_id", axis=1)
dfs["avg_teams_stats"] = dfs["avg_teams_stats"].reset_index()

# Drop winRate from per-game stats as it is only meaningful as a mean in the average stats
dfs["game_teams_stats"] = dfs["game_teams_stats"].drop("winRate", axis=1)

In [15]:
# Calculating Corsi and Fenwick per game
# Self-join to allow cross-comparison of shots
cols = ["game_id", "team_id", "shots", "blocked"]
df_a = dfs["game_teams_stats"][cols].rename(columns={"team_id": "team_id_a"})
df_b = dfs["game_teams_stats"][cols].rename(columns={"team_id": "team_id_b"})
df = dd.merge(df_a, df_b, on="game_id", suffixes=("_a", "_b"))
# Cleanup duplicates after self-join
df = df[df["team_id_a"] != df["team_id_b"]]
# Calculate Corsi and Fenwick
df["corsi_a"] = df["shots_a"] - df["shots_b"]
df["fenwick_a"] = df["shots_a"] - df["blocked_a"] - (df["shots_b"] - df["blocked_b"])
# Use smallest int that fits the min and max to save memory
df["corsi_a"] = df["corsi_a"].astype("int8[pyarrow]")
df["fenwick_a"] = df["fenwick_a"].astype("int8[pyarrow]")
# Cleanup redundant columns
cols = ["shots_a", "blocked_a", "team_id_b", "shots_b", "blocked_b"]
df = df.drop(cols, axis=1)
# Remove suffixes
cols = {"team_id_a": "team_id", "corsi_a": "corsi", "fenwick_a": "fenwick"}
df = df.rename(columns=cols)
# Save to dataframe collection
dfs["game_teams_corfen"] = df

# Compute the average Corsi and Fenwick per team and save to avg_teams_stats
df = df.groupby("team_id").mean(numeric_only=True)
df = df.reset_index()
df["team_id"] = df["team_id"].astype("int64[pyarrow]")
df = df.drop("game_id", axis=1)
dfs["avg_teams_stats"] = dfs["avg_teams_stats"].join(df, on="team_id", rsuffix="_right")
dfs["avg_teams_stats"] = dfs["avg_teams_stats"].drop("team_id_right", axis=1)

In [16]:
# Generate average skater stats from per-game skater stats
dfs["avg_skater_stats"] = (
    dfs["game_skater_stats"].groupby("player_id").mean(numeric_only=True)
)
dfs["avg_skater_stats"] = dfs["avg_skater_stats"].drop(["game_id", "team_id"], axis=1)
dfs["avg_skater_stats"] = dfs["avg_skater_stats"].reset_index()

In [17]:
# Convert the decision string to int to allow the mean to calculate the win rate
def decision_to_num(decision):
    if isinstance(decision, str):
        if decision == "W":
            return 1
        elif decision == "L":
            return 0
        else:
            raise Exception(f"Unexpected string '{decision}'")
    else:
        return np.nan


dfs["game_goalie_stats"]["decisionWinRate"] = dfs["game_goalie_stats"][
    "decision"
].apply(decision_to_num, meta=(0, "int8[pyarrow]"))

# Generate average goalie stats from per-game goalie stats
dfs["avg_goalie_stats"] = (
    dfs["game_goalie_stats"].groupby("player_id").mean(numeric_only=True)
)
dfs["avg_goalie_stats"] = dfs["avg_goalie_stats"].drop(["game_id", "team_id"], axis=1)
dfs["avg_goalie_stats"] = dfs["avg_goalie_stats"].reset_index()

# Drop the redundant column from per-game stats
dfs["game_goalie_stats"] = dfs["game_goalie_stats"].drop("decisionWinRate", axis=1)

In [18]:
# Calculate the geometric mean of positive-meaning stats in avg_teams_stats
# Note that we cannot include the Corsi and Fenwick as they have negative values
cols = [
    "goals",
    "shots",
    "hits",
    "powerPlayOpportunities",
    "powerPlayGoals",
    "faceOffWinPercentage",
    "takeaways",
    "blocked",
]
dfs["avg_teams_stats"]["gmean_stats"] = gmean(dfs["avg_teams_stats"], cols)

In [19]:
# Calculate the geometric mean of positive-meaning stats in avg_skater_stats
cols = [
    "timeOnIce",
    "assists",
    "goals",
    "shots",
    "hits",
    "powerPlayGoals",
    "powerPlayAssists",
    "penaltyMinutes",
    "faceOffWins",
    "faceoffTaken",
    "takeaways",
    "shortHandedGoals",
    "shortHandedAssists",
    "blocked",
    "plusMinus",
    "evenTimeOnIce",
    "shortHandedTimeOnIce",
    "powerPlayTimeOnIce",
]
dfs["avg_skater_stats"]["gmean_stats"] = gmean(dfs["avg_skater_stats"], cols)

In [20]:
# Calculate the geometric mean of positive-meaning stats in avg_goalie_stats
cols = [
    "timeOnIce",
    "assists",
    "goals",
    "pim",
    "shots",
    "saves",
    "powerPlaySaves",
    "shortHandedSaves",
    "evenSaves",
    "shortHandedShotsAgainst",
    "evenShotsAgainst",
    "powerPlayShotsAgainst",
    "savePercentage",
    "powerPlaySavePercentage",
    "evenStrengthSavePercentage",
]
dfs["avg_goalie_stats"]["gmean_stats"] = gmean(dfs["avg_goalie_stats"], cols)

# Load


In [ ]:
# Export to parquet for future manipulations
for k in dfs:
    fpath = os.path.join(EXPORT_PATH, k + ".parquet")
    dfs[k].to_parquet(fpath, compression="zstd", write_index=True)
    print(f"📂 exported {fpath}")

In [ ]:
# Load the tables into an sql server
# note that it takes about an hour due to data size
def load_with_progress(df: dd.DataFrame, table_name: str, uri: str) -> None:
    with tqdm(total=df.npartitions, desc=table_name) as pbar:
        for i in range(df.npartitions):
            if_exists = "replace" if i == 0 else "append"
            df.get_partition(i).to_sql(
                table_name,
                uri,
                index=True,
                if_exists=if_exists,
                chunksize=CHUNKSIZE,
                method="multi",
            )
            pbar.update(1)


for k in dfs:
    load_with_progress(dfs[k], k, sql_uri)